In [2]:
conda install -c pytorch torchtext

==> WARNING: A newer version of conda exists. <==


Solving environment: ...working... done


Note: you may need to restart the kernel to use updated packages.

  current version: 4.8.3

  latest version: 4.9.0

...

  environment location: C:\Users\Mohit\anaconda3\envs\venv

  added / updated specs:

    $ conda update -n base -c defaults conda

    - torchtext




The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    torchtext-0.7.0            |             py38         2.8 MB  pytorch
    ------------------------------------------------------------
                                           Total:         2.8 MB

The following NEW packages will be INSTALLED:

  torchtext          pytorch/win-64::torchtext-0.7.0-py38




torchtext-0.7.0      | 2.8 MB    |            |   0% 
torchtext-0.7.0      | 2.8 MB    |            |   1% 
torchtext-0.7.0      | 2.8 MB    | 1          |   2% 
torchtext-0.7.0      | 2.8 MB    | 3          |   4% 
torchtext-0.7.0      | 2.8 MB    | 8          |   8% 
torchtext-0.7.0      | 2.8 MB    | #2         |  13% 
torchtext-0.7.0      | 2.8 MB    | ##1        |  21% 
torchtext-0.7.0      | 2.8 MB    | ###9       |  39% 
torchtext-0.7.0      | 2.8 MB    | ####4      |  44% 
torchtext-0.7.0      | 2.8 MB    | #######4   |  75% 

In [14]:
exec('from __future__ import unicode_literals')

import os
import sys
import random

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../onmt'))
if module_path not in sys.path:
    sys.path.append(module_path)

from itertools import repeat

from onmt.utils.logging import init_logger
from onmt.utils.misc import split_corpus
from onmt.translate.translator import build_translator

import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
from kp_gen_eval import _get_parser


from nltk.corpus import stopwords
stoplist = stopwords.words('english')

from string import punctuation
import onmt.keyphrase.pke as pke
from nltk.corpus import stopwords

import onmt.keyphrase.kp_inference as kp_inference

In [15]:
import importlib
importlib.reload(kp_inference)

<module 'onmt.keyphrase.kp_inference' from 'D:\\OpenNMT-kpg-release\\onmt\\keyphrase\\kp_inference.py'>

### Load a text (assume current directory is OpenNMT-kpg/notebook/)

In [31]:
data_root_path = 'D:\\OpenNMT-kpg-release\\test'
print(os.path.abspath(data_root_path))
doc_dicts = []
for subdir, dirs, files in os.walk(data_root_path):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".txt"):
#             print(filepath)
            text = open(filepath, 'r').readlines()
            pars = [l for l in text if len(l.split()) > 20]
            doc = {'name': file, 'path': filepath, 'text': text, 'paragraph': pars}
#             print('#line=%d, #par=%d' % (len(text), len(pars)))
            doc_dicts.append(doc)
    
print('Loaded #(docs)=%d' % (len(doc_dicts)))

D:\OpenNMT-kpg-release\test
Loaded #(docs)=1


##### Sample a paragraph

1

In [43]:
doc_id = random.randint(0, len(doc_dicts)-1)
doc = doc_dicts[doc_id]
pars = doc['paragraph']
par_id = random.randint(0, len(doc_dicts[doc_id]['paragraph'])-1)
text_to_extract = pars[par_id]
print(doc_id)
print(par_id)
print(text_to_extract)


0
0
It is the most complex and original idea I have ever seen or heard of. Also, because it delves into the topic of human emotions, but in an artificial way, if I were to be punny and serious all at once. From what I have seen of the show, I believe that we, as human beings, can compare ourselves to the androids, because we can definitely relate to them


### Supervised Deep Keyphrase Model

In [53]:
parser = _get_parser()
config_path = '../config/translate/config-rnn-keyphrase.yml'
one2one_ckpt_path = '../models/keyphrase/meng17-one2one-kp20k-topmodels/kp20k-meng17-one2one-rnn-BS128-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Covfalse-Contboth-IF1_step_30000.pt'
one2seq_ckpt_path = '../models/keyphrase/meng17-one2seq-kp20k-topmodels/kp20k-meng17-verbatim_append-rnn-BS64-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Reusetrue-Covtrue-PEfalse-Contboth-IF1_step_50000.pt'
opt = parser.parse_args('-config %s' % (config_path))
setattr(opt, 'models', [one2one_ckpt_path])

translator = build_translator(opt, report_score=False)

In [54]:
scores, predictions = translator.translate(
    src=[text_to_extract],
    tgt=None,
    src_dir=opt.src_dir,
    batch_size=opt.batch_size,
    attn_debug=opt.attn_debug,
    opt=opt
)
print('Paragraph:\n\t'+text_to_extract)
print('Top predictions:')
keyphrases = [kp.strip() for kp in predictions[0] if (not kp.lower().strip() in stoplist) and (kp != '<unk>' )]
for kp_id, kp in enumerate(keyphrases[: min(len(keyphrases), 20)]):
    print('\t%d: %s' % (kp_id+1, kp))

AttributeError: 'str' object has no attribute 'decode'

### PKE models

#### TF-IDF

In [3]:
dataset_name = 'test'
dataset_path = '../data/%s/' % dataset_name
_ = kp_inference.extract_pke(text_to_extract, method='tfidf' , dataset_path=dataset_path,
            df_path=os.path.abspath(dataset_path + '../%s.df.tsv.gz' % dataset_name), top_k=20)

	1: anything (0.0000)
	2: anything that (0.0000)
	3: anything that threatens (0.0000)
	4: that (0.0000)
	5: that threatens (0.0000)
	6: threatens (0.0000)
	7: company (0.0000)
	8: ability (0.0000)
	9: ability to (0.0000)
	10: ability to meet (0.0000)
	11: to meet (0.0000)
	12: to meet its (0.0000)
	13: meet (0.0000)
	14: meet its (0.0000)
	15: meet its target (0.0000)
	16: its (0.0000)
	17: its target (0.0000)
	18: its target or (0.0000)
	19: target (0.0000)
	20: target or (0.0000)


#### YAKE

In [4]:
_ = kp_inference.extract_pke(text_to_extract, method='yake', top_k=20)

	1: called business risk (0.0331)
	2: financial goals (0.0332)
	3: called business (0.0632)
	4: business risk (0.0830)
	5: risk (0.1075)
	6: company (0.1209)
	7: anything (0.1383)
	8: ability (0.1383)
	9: risks may come (0.1415)
	10: business (0.1657)
	11: may (0.1765)
	12: threatens (0.1793)
	13: meet (0.1793)
	14: target (0.1793)
	15: achieve (0.1793)
	16: financial (0.1793)
	17: goals (0.1793)
	18: called (0.1793)
	19: risk management strategy (0.1844)
	20: anything that threatens (0.1851)


#### TextRank

In [5]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

	1: risk management (0.1901)
	2: company head (0.1464)
	3: financial goals (0.1464)
	4: risk (0.0950)
	5: sources (0.0732)
	6: company (0.0732)
